In [26]:
import json
import pandas as pd
import folium
from matplotlib import colormaps
from matplotlib.colors import rgb2hex


In [27]:
with open('data/booking_hotels.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Grouper les données par ville
grouped_data = {}
for item in data:
    city = item['city']
    if city not in grouped_data:
        grouped_data[city] = []
    grouped_data[city].append(item)

csv_filename = 'data/hotels.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    # Définir les en-têtes du fichier CSV
    fieldnames = ['city', 'hotel_name', 'hotel_address', 'latitude', 'longitude', 'hotel_rating', 'hotel_description', 'hotel_url']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier CSV
    writer.writeheader()

    # Écrire les informations des hôtels dans le fichier CSV, classées par ville
    for city, hotels in grouped_data.items():
        for hotel in hotels:
            writer.writerow({
                'city': city,
                'hotel_name': hotel['hotel_name'],
                'hotel_address': hotel['hotel_address'],
                'latitude' : hotel['latitude'],
                'longitude' : hotel['longitude'],
                'hotel_rating': hotel['hotel_rating'],
                'hotel_description': hotel['hotel_description'],
                'hotel_url': hotel['hotel_url']
            })

In [28]:
hotels = pd.read_csv("data/hotels.csv")
hotels

,city,hotel_name,hotel_address,latitude,longitude,hotel_rating,hotel_description,hotel_url
0,Bormes les Mimosas,Appartement pieds dans l’eau Vue Mer Bormes Le...,"13 Boulevard du Front de Mer, 83230 Bormes-les...",43.122153,6.359464,9.1,L’hébergement Appartement pieds dans l’eau Vue...,https://www.booking.com/hotel/fr/appartement-p...
1,Bormes les Mimosas,SELECT'SO HOME - Résidence Le Poséidon - Mazet...,"7 Avenue de la Mer, 83230 Bormes-les-Mimosas, ...",43.124150,6.351192,8.4,L’hébergement SELECT'SO HOME - Résidence Le Po...,https://www.booking.com/hotel/fr/poseidon-maze...
2,Bormes les Mimosas,Bel appartement classé 3 étoiles,"100 Boulevard du Mont des Roses, 83230 Bormes-...",43.143983,6.337272,9.5,L’hébergement 3 étoiles Bel appartement classé...,https://www.booking.com/hotel/fr/joli-2-pieces...
3,Bormes les Mimosas,Hotel La Voile,"165 Avenue Des Girelles - La Favière, 83230 Bo...",43.125490,6.357300,8.7,L'Hotel La Voile est idéalement situé à seulem...,https://www.booking.com/hotel/fr/hotel-la-voil...
4,Bormes les Mimosas,Studio avec Jardin et Parking Privés,"217 Rue des Iris, 83230 Bormes-les-Mimosas, Fr...",43.140363,6.336715,8.9,L’hébergement Studio avec Jardin et Parking Pr...,https://www.booking.com/hotel/fr/217-rue-des-i...
...,...,...,...,...,...,...,...,...
119,Nimes,SQUARE HOTEL,"7 Square de la Couronne, 30000 Nîmes, France",43.837032,4.363037,8.6,L’établissement SQUARE HOTEL vous accueille à ...,https://www.booking.com/hotel/fr/square-nimes....
120,Nimes,Comfort Aparthotel Nimes Centre,"5 allée Boissy d'Anglas - Triangle de la Gare,...",43.832493,4.367857,8.1,Le Comfort Aparthotel Nîmes Centre est une rés...,https://www.booking.com/hotel/fr/adagio-access...
121,Nimes,B&B HOTEL Nîmes Centre Arènes,"83, rue de la République, 30900 Nîmes, France",43.828161,4.354790,8.4,"Situé à Nîmes, à 950 mètres des arènes, le B&B...",https://www.booking.com/hotel/fr/b-amp-b-nimes...
122,Nimes,Appart'City Collection Nîmes Arènes,"1 Boulevard De Bruxelles, 30000 Nîmes, France",43.833874,4.361434,8.9,L'Appart'City Collection Nîmes Arènes vous acc...,https://www.booking.com/hotel/fr/appart-39-cit...


In [29]:
#les nouveaux hotels n'ont pas encore de notation sur booking:
hotels['hotel_rating'] = pd.to_numeric(hotels['hotel_rating'], errors='coerce')
hotels['hotel_rating'] = hotels['hotel_rating'].fillna("Nouvel établissement sur Booking. Pas encore de notation")

In [30]:
hotels["city"].value_counts()

city
La Rochelle           25
Marseille             25
Mont Saint Michel     25
Nimes                 25
Bormes les Mimosas    24
Name: count, dtype: int64

In [31]:
print("min:",hotels["hotel_rating"].min())
print("max:",hotels["hotel_rating"].max())
print(hotels["hotel_rating"].value_counts())

min: 7.1
max: 9.7
hotel_rating
8.3    11
9.3    11
8.9     9
8.6     9
9.1     8
8.7     6
8.8     6
9.0     6
8.2     5
9.5     5
7.7     5
7.8     5
7.9     4
8.1     4
8.4     4
9.6     4
9.2     3
7.5     3
9.7     3
7.4     2
9.4     2
8.5     2
7.3     2
8.0     2
7.6     1
7.1     1
7.2     1
Name: count, dtype: int64


In [32]:
# Fonction pour convertir une note en couleur entre rouge (0) et vert (10)
def rating_to_color(rating, min_rating=7, max_rating=10):
    try:
        rating = float(rating)
        cmap = colormaps['RdYlGn']
        normalized = (rating - min_rating) / (max_rating - min_rating)
        rgba = cmap(normalized)
        return rgb2hex(rgba)
    except (ValueError, TypeError):
        return '#999999'  # Gris pour note absente / non numérique

# Créer une carte, initialisation vide
map = folium.Map(location=[0, 0], zoom_start=2)
# Liste pour stocker les coordonnées
bounds = []

# Markers avec couleurs dynamiques
for _, row in hotels.iterrows():
    lat, lon = row['latitude'], row['longitude']
    bounds.append([lat, lon])
    color = rating_to_color(row['hotel_rating'])

    folium.CircleMarker(
        location=[lat, lon],
        radius=7,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=folium.Popup(
            f"<b>{row['hotel_name']}</b><br>{row['hotel_address']}<br><b>Note:</b> {row['hotel_rating']}<br><br>{row['hotel_description']}<br><a href='{row['hotel_url']}' target='_blank'>Voir</a>",
            max_width=300
        )
    ).add_to(map)

# Centrer la carte sur tous les markers
map.fit_bounds(bounds)


map.save('maps/best_hotels.html')

# Afficher la carte
map


In [33]:
# Enregistrer le DataFrame en fichier CSV
hotels.to_csv('data/hotels.csv', index=False)